<h1><center>Analysis of new media on stock returns</center><h1>

## Project Team – Group 13
- Vidur  Puliani          A0198492L
- Lei Xiaowen             A0198449H
- Donal Ngo Jin Ze        A0198487A
- Wilson Lum Kok Keong    A0198478A
<br><br>

## To scrape Twitter data for the 10 technology stosks name as follow :
- Apple = NASDAQ: AAPL
- Google = NASDAQ: GOOGL
- Facebook = NASDAQ: FB
- Amazon = NASDAQ: AMZN
- Microsoft = NASDAQ: MSFT
- Alibaba = NYSE: BABA
- AMD = NASDAQ: AMD
- Intel = NASDAQ: INTC
- Tesla = NASDAQ: TSLA
- DOw Jones = DIA
- SPider = NSADAQ: SPY
- Tweeter = TWTR

## Twitter Authenication credential

In [3]:
import pandas as pd
import re
from datetime import date
import tweepy
import time
import sys
from collections import Counter
from textblob import TextBlob

# Twitter account API credential details
consumer_key = "L8KaOD3tPINfmV74ZvQv7onCV" 
consumer_secret = "HPvvTrEVPtr7yTsOnSSOZ2XOIHxa4Kk0yrTKyFosgdPLO927i1"
access_token = "1201691382752014337-afzYJEq3tQNNwFg4TVnh0yToZX6Cnj"
access_token_secret = "wm8zwYtlc47tWnl8Ah8nzaSE7qDAeUpF1wUymC2qpVMJ7"

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.careated_at)


## Define Variables

In [3]:
search_query = ['$AAPL','$goog','$FB ','$amzn','$MSFT','$baba','$AMD', '$intc','$tsla','$dia','$spy','$twtr','$vix']

maxTweets    = 100000

### Define Functions

In [6]:
# Collect tweets per stocks
# Favorite_count => A tweet that is favorited
#                   The favorite_count provides the number of times the tweet has been favorited. 
#                   In the case of a retweet, favorite_count is the favorite count of the source tweet
# retweet_count  => A Retweet is a re-posting of a Tweet. Twitter's Retweet feature helps you and others quickly share that  
#                   Tweet with all of your followers. You can Retweet your own Tweets or Tweets from someone else. 
#                   Sometimes people type "RT" at the beginning of a Tweet to indicate that they are re-posting 
#                   someone else's content.

def countdown(t, step=1, msg='sleeping'):  # in seconds
    pad_str = ' ' * len('%d' % step)
    for i in range(t, 0, -step):
        print('\r%s for the next %d seconds %s\r' % (msg, i, pad_str),end='\r')
        time.sleep(step)
    print('Done %s for %d seconds! ' % (msg, t))
    
def collect_tweets(search_query):
    rest         = 900 # if rating error wait for 15 mins - Twitter limit
    my_tweets    = []
    maxTweets    = 100000 # Some arbitrary large number
    tweetsPerQry = 100   # this is the max the API permits
    
    # If results from a specific ID onwards are reqd, set since_id to that ID.
    # else default to no lower limit, go as far back as API allows
    sinceId = None
    
    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1
    
    tweetCount = 0
    
    print("Downloading max {0} tweets".format(maxTweets))
    print ("Collecting tweets for   :" + str(search_query))
    
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=search_query, count=tweetsPerQry,lang='en',include_entities=True)
                else:
                    new_tweets = api.search(q=search_queryy, count=tweetsPerQry,lang='en',since_id=sinceId,
                                            include_entities=True)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=search_query, count=tweetsPerQry,lang='en',max_id=str(max_id - 1),
                                            include_entities=True)
                else:
                    new_tweets = api.search(q=search_query, count=tweetsPerQry,lang='en',max_id=str(max_id - 1),
                                            include_entities=True,since_id=sinceId)
       
            if not new_tweets:
                print("No more tweets found")
                break
            
            for tweet in new_tweets:
                my_tweets.append([tweet.created_at, tweet.text,tweet.favorite_count,tweet.retweet_count,search_query])
            
            tweetCount += len(new_tweets)
            
            if (tweetCount % 1000==0):
                print("Downloaded {0} tweets".format(tweetCount))
         
            max_id = new_tweets[-1].id
            
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            print("Sleeping ... for error ... {0}s ...".format(rest))
            
            countdown(rest)
            
            if (tweetCount % 1000==0):
                print("Downloaded {0} tweets".format(tweetCount))

    print ("Downloaded {0} tweets".format(tweetCount))
    
    return my_tweets

# Preprocess the tweets
def pre_process_tweets(my_tweets,search_query):
    processed_tweets = []
    tweets_processed =my_tweets
    print ("Preprocessing texts for :" + str(search_query))
    
    for tweet in range(0, len(my_tweets)):  
    
        # Remove all the http links
        processed_tweet = re.sub(r'http\S+', '', str(my_tweets[tweet][1]))
        
        # Remove all the special characters
        processed_tweet = re.sub(r'\W', ' ', processed_tweet)
 
        # Substituting multiple spaces with single space
        processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
        # Removing prefixed 'b'
        processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
        # Converting to Lowercase
        processed_tweet = processed_tweet.lower()
        
        tweets_processed[tweet][1] = processed_tweet
       
    return tweets_processed


def Remove(my_tweets,search_query): 
    
    print ("Removing dulpicates texts for :" + str(search_query))
    text = []
    new_list = []
    for tweet in range(0, (len(my_tweets))):  
        if tweet < (len(my_tweets)-1):
            if my_tweets[tweet][1] not in text:
                text.append(my_tweets[tweet][1])
                new_list.append(my_tweets[tweet]) 
            
    return new_list
        

## Collecting Tweets and Preprocess the data

In [8]:
# List of stocks to collect for tweets
my_tweets_df = [ [0] * maxTweets for _ in range(len(search_query))]
header = ['Date', 'Text', 'favorite_count', 'retweet_count','ticker']

for i in range (0,len(search_query)):
    my_tweets       = collect_tweets(search_query[i])
    my_tweets_1     = pre_process_tweets(my_tweets,search_query[i])
    my_tweets_2     = Remove(my_tweets_1,search_query[i])
    my_tweets_df[i] = pd.DataFrame(my_tweets_2)
    
    if not my_tweets_df[i].empty:
        my_tweets_df[i].columns = header
         
    print("\n------------------------------------------\n")
    
print("Done")

Downloaded 1000 tweets
Downloaded 2000 tweets
Downloaded 3000 tweets
Downloaded 4000 tweets
Downloaded 5000 tweets
No more tweets found
Downloaded 16137 tweets
Preprocessing texts for :$AAPL
Removing dulpicates texts for :$AAPL

------------------------------------------

Downloaded 1000 tweets
some error : [{'message': 'Rate limit exceeded', 'code': 88}]
Sleeping ... for error ... 900s ...
sleeping for the next 1 seconds    Done sleeping for 900 seconds! 
Downloaded 2000 tweets
No more tweets found
Downloaded 2434 tweets
Preprocessing texts for :$goog
Removing dulpicates texts for :$goog

------------------------------------------

Downloaded 1000 tweets
Downloaded 2000 tweets
Downloaded 3000 tweets
No more tweets found
Downloaded 6489 tweets
Preprocessing texts for :$FB 
Removing dulpicates texts for :$FB 

------------------------------------------

Downloaded 1000 tweets
Downloaded 2000 tweets
Downloaded 3000 tweets
No more tweets found
Downloaded 9352 tweets
Preprocessing texts fo

In [9]:
Stock_counter = Counter()

for i in range(0,len(my_tweets_df)):
    Stock_counter[search_query[i]] = len(my_tweets_df[i])
    print("Number of tweets for " + search_query[i] + " is " + str(len(my_tweets_df[i])))

Stock_counter.most_common()

Number of tweets for $AAPL is 10438
Number of tweets for $goog is 1763
Number of tweets for $FB  is 4103
Number of tweets for $amzn is 5923
Number of tweets for $MSFT is 5163
Number of tweets for $baba is 1918
Number of tweets for $AMD is 3028
Number of tweets for $intc is 916
Number of tweets for $tsla is 18574
Number of tweets for $dia is 3035
Number of tweets for $spy is 30253
Number of tweets for $twtr is 2497
Number of tweets for $vix is 5502


[('$spy', 30253),
 ('$tsla', 18574),
 ('$AAPL', 10438),
 ('$amzn', 5923),
 ('$vix', 5502),
 ('$MSFT', 5163),
 ('$FB ', 4103),
 ('$dia', 3035),
 ('$AMD', 3028),
 ('$twtr', 2497),
 ('$baba', 1918),
 ('$goog', 1763),
 ('$intc', 916)]

## Save the datasets to Excel file

In [10]:
#*******************************************************************************
#                         Save data
#*******************************************************************************

# Month abbreviation, day and year
today = date.today()
d = today.strftime("%b-%d-%Y")

# Create a new excel workbook
writer = pd.ExcelWriter('data/stock_tweets_' + d + '.xlsx', engine='xlsxwriter')

for i in range (0,len(search_query)):
    my_tweets_df[i].to_excel(writer, sheet_name= search_query[i], index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()